# 5分钟使用Python爬取豆瓣TOP250电影榜

本视频的演示步骤：

1. 使用requests爬取网页
2. 使用BeautifulSoup实现数据解析
3. 借助pandas将数据写出到Excel

这三个库的详细用法，请看我的其他视频课程

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 1、下载共10个页面的HTML

In [4]:
# 构造分页数字列表
page_indexs = range(0, 250, 25)

In [5]:
list(page_indexs)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225]

In [6]:
def download_all_htmls():
    """
    下载所有列表页面的HTML，用于后续的分析
    """
    htmls = []
    for idx in page_indexs:
        url = f"https://movie.douban.com/top250?start={idx}&filter="
        print("craw html:", url)
        r = requests.get(url,
                        headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"})
        if r.status_code != 200:
            raise Exception("error")
        htmls.append(r.text)
    return htmls

In [7]:
# 执行爬取
htmls = download_all_htmls()

craw html: https://movie.douban.com/top250?start=0&filter=
craw html: https://movie.douban.com/top250?start=25&filter=
craw html: https://movie.douban.com/top250?start=50&filter=
craw html: https://movie.douban.com/top250?start=75&filter=
craw html: https://movie.douban.com/top250?start=100&filter=
craw html: https://movie.douban.com/top250?start=125&filter=
craw html: https://movie.douban.com/top250?start=150&filter=
craw html: https://movie.douban.com/top250?start=175&filter=
craw html: https://movie.douban.com/top250?start=200&filter=
craw html: https://movie.douban.com/top250?start=225&filter=


In [8]:
htmls[0]

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-windows ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n豆瓣电影 Top 250\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">\n    \n    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">\n    <link href="https://img3.doubanio.com/f/shire/3e5dfc68b0f376484c50cf08a58bbca3700911dc/css/douban.css" rel="stylesheet" type="text/css">\n    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="style

## 2、解析HTML得到数据

In [9]:
def parse_single_html(html):
    """
    解析单个HTML，得到数据
    @return list({"link", "title", [label]})
    """
    soup = BeautifulSoup(html, 'html.parser')
    article_items = (
        soup.find("div", class_="article")
            .find("ol", class_="grid_view")
            .find_all("div", class_="item")
    )
    datas = []
    for article_item in article_items:
        rank = article_item.find("div", class_="pic").find("em").get_text()
        info = article_item.find("div", class_="info")
        title = info.find("div", class_="hd").find("span", class_="title").get_text()
        stars = (
            info.find("div", class_="bd")
                .find("div", class_="star")
                .find_all("span")
        )
        rating_star = stars[0]["class"][0]
        rating_num = stars[1].get_text()
        comments = stars[3].get_text()
        
        datas.append({
            "rank":rank,
            "title":title,
            "rating_star":rating_star.replace("rating","").replace("-t",""),
            "rating_num":rating_num,
            "comments":comments.replace("人评价", "")
        })
    return datas



In [8]:
import pprint
pprint.pprint(parse_single_html(htmls[0]))

[{'comments': '1790062',
  'rank': '1',
  'rating_num': '9.7',
  'rating_star': '5',
  'title': '肖申克的救赎'},
 {'comments': '1320110',
  'rank': '2',
  'rating_num': '9.6',
  'rating_star': '5',
  'title': '霸王别姬'},
 {'comments': '1376257',
  'rank': '3',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '阿甘正传'},
 {'comments': '1570867',
  'rank': '4',
  'rating_num': '9.4',
  'rating_star': '45',
  'title': '这个杀手不太冷'},
 {'comments': '879501',
  'rank': '5',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '美丽人生'},
 {'comments': '1314428',
  'rank': '6',
  'rating_num': '9.4',
  'rating_star': '45',
  'title': '泰坦尼克号'},
 {'comments': '1407622',
  'rank': '7',
  'rating_num': '9.3',
  'rating_star': '45',
  'title': '千与千寻'},
 {'comments': '707102',
  'rank': '8',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '辛德勒的名单'},
 {'comments': '1337018',
  'rank': '9',
  'rating_num': '9.3',
  'rating_star': '45',
  'title': '盗梦空间'},
 {'comments': '909227',
  'rank': '10',
  'ra

In [10]:
# 执行所有的HTML页面的解析
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(html))

In [11]:
all_datas

[{'rank': '1',
  'title': '肖申克的救赎',
  'rating_star': '5',
  'rating_num': '9.7',
  'comments': '1999723'},
 {'rank': '2',
  'title': '霸王别姬',
  'rating_star': '5',
  'rating_num': '9.6',
  'comments': '1480659'},
 {'rank': '3',
  'title': '阿甘正传',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '1514680'},
 {'rank': '4',
  'title': '这个杀手不太冷',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1708218'},
 {'rank': '5',
  'title': '美丽人生',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '954447'},
 {'rank': '6',
  'title': '泰坦尼克号',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1464216'},
 {'rank': '7',
  'title': '千与千寻',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1566664'},
 {'rank': '8',
  'title': '辛德勒的名单',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '772666'},
 {'rank': '9',
  'title': '盗梦空间',
  'rating_star': '45',
  'rating_num': '9.3',
  'comments': '1450577'},
 {'rank': '10',
  'title': '忠犬八公的故事',
  'rati

In [12]:
len(all_datas)

250

## 3、将结果存入excel

In [13]:
df = pd.DataFrame(all_datas)

In [14]:
df

comments rank rating_num rating_star         title
0    1999723    1        9.7           5        肖申克的救赎
1    1480659    2        9.6           5          霸王别姬
2    1514680    3        9.5           5          阿甘正传
3    1708218    4        9.4          45       这个杀手不太冷
4     954447    5        9.5           5          美丽人生
5    1464216    6        9.4          45         泰坦尼克号
6    1566664    7        9.4          45          千与千寻
7     772666    8        9.5           5        辛德勒的名单
8    1450577    9        9.3          45          盗梦空间
9    1005507   10        9.4          45       忠犬八公的故事
10   1208661   11        9.3          45         海上钢琴师
11   1074274   12        9.3          45         楚门的世界
12   1342348   13        9.2          45       三傻大闹宝莱坞
13    956385   14        9.3          45        机器人总动员
14    934322   15        9.3          45        放牛班的春天
15   1079377   16        9.3          45          星际穿越
16   1058463   17        9.2          45     大话西游之大圣娶亲
17    651504   18        9.3          45            熔炉
18   1261417   19        9.2          45         疯狂动物城
19    861688   20        9.2          45           无间道
20    898589   21        9.2          45            龙猫
21    656386   22        9.3          45            教父
22   1073035   23        9.1          45        当幸福来敲门
23   1247244   24        9.1          45          怦然心动
24    701635   25        9.2          45          触不可及
25    722541   26        9.2          45      蝙蝠侠：黑暗骑士
26    276617   27        9.6           5          控方证人
27    560295   28        9.2          45            活着
28    480612   29        9.3          45          乱世佳人
29   1051205   30        9.1          45         寻梦环游记
..       ...  ...        ...         ...           ...
220   147140  221        8.9          45        地球上的星星
221   306603  222        8.6          45          谍影重重
222   258121  223        8.6          45          战争之王
223   248789  224        8.7          45            血钻
224   336149  225        8.5          45           猜火车
225   472502  226        8.4           4           色，戒
226   258428  227        8.6          45          遗愿清单
227   265471  228        8.7          45         大佛普拉斯
228   361691  229        8.6          45  疯狂的麦克斯4：狂暴之路
229   176890  230        8.8          45          步履不停
230   352045  231        8.5          45       彗星来的那一夜
231   366794  232        8.6          45           朗读者
232   200411  233        8.7          45            浪潮
233   348149  234        8.4           4      小萝莉的猴神大叔
234   297507  235        8.5          45    再次出发之纽约遇见你
235   205676  236        8.8          45            聚焦
236   649807  237        8.3           4           驴得水
237    89291  238        9.2          45          东京物语
238   129345  239        8.9          45            追随
239   193821  240        8.7          45          一次别离
240   366720  241        8.5          45         九品芝麻官
241   169502  242        8.8          45          千钧一发
242   107341  243        9.0          45           我爱你
243   205835  244        8.7          45          黑鹰坠落
244   339342  245        8.5          45     哈利·波特与火焰杯
245   115878  246        8.9          45          四个春天
246   373854  247        8.6          45          网络谜踪
247   276588  248        8.5          45           发条橙
248   254926  249        8.6          45      E.T. 外星人
249   250278  250        8.6          45    黑客帝国2：重装上阵

[250 rows x 5 columns]

In [16]:
df.to_excel("豆瓣电影TOP250.xlsx")